# Aula 10 - Recomendação baseada em sessão - Exercícios

In [2]:
import pandas as pd
import numpy as np

### Leitura do arquivo 2019-Oct-sample.csv (vide Aula 10 - Exemplos) caso não possua o arquivo

In [18]:
!unzip 2019-Oct-sample.csv.zip

Archive:  2019-Oct-sample.csv.zip
  inflating: 2019-Oct-sample.csv     


In [19]:
subset = pd.read_csv('./2019-Oct-sample.csv')
subset.head()

,event_time,event_type,product_id,category_code,brand,user_session
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c


In [20]:
map_items = {item: idx for idx, item in enumerate(subset.product_id.unique())}
map_sessions = {item: idx for idx, item in enumerate(subset.user_session.unique())}
subset['itemId'] = subset['product_id'].map(map_items)
subset['sessionId'] = subset['user_session'].map(map_sessions)
subset.head()

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,0,0
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c,2,0
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,3,0


In [21]:
n_items = subset['itemId'].max()+1
print('No. items: ', n_items)
n_sessions = subset['sessionId'].max()+1
print('No. sessions: ', n_sessions)

No. items:  42581
No. sessions:  483508


In [22]:
# create a dataset
# remove sessions with less than 2 items
def create_data(df):
    df.sort_values(by=['sessionId', 'event_time'], inplace=True, ignore_index=True)
    sessions, session = [], []
    for index, value in df.iterrows():
        if index != 0:
            if value["sessionId"] == df.at[index-1, "sessionId"]:
                if value["event_type"] == 'view':
                    session.append(value["itemId"])
            else:
                if len(session) > 1:
                    sessions.append((df.at[index-1, "sessionId"], session))
                session = [value["itemId"]]
        else:
            session.append(value["itemId"])
    return sessions

In [23]:
sessions = create_data(subset)

In [24]:
import random

random.shuffle(sessions)
split = len(sessions) * 0.8
train = sessions[:int(split)]
test = sessions[int(split):]
print('No. train sessions: ', len(train))
print('No. test sessions: ', len(test))

No. train sessions:  237531
No. test sessions:  59383


In [25]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [26]:
actual_session = test[3]
target = actual_session[1][0:-1]
print(actual_session)
print(target)
subset.loc[subset.sessionId==actual_session[0]]

(6372, [1796, 1796])
[1796]


,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
25206,2019-10-12 04:29:54 UTC,view,1004669,electronics.smartphone,xiaomi,004184e4-8a9c-433c-be3a-684f858649ae,1796,6372
25207,2019-10-12 04:30:36 UTC,view,1004669,electronics.smartphone,xiaomi,004184e4-8a9c-433c-be3a-684f858649ae,1796,6372


In [27]:
actual_session[1][-1]

1796

In [28]:
train

[(401844, [3132, 3132]),
 (184492, [1949, 1949, 1949, 1949]),
 (611, [989, 990, 991, 992, 992, 992, 992]),
 (39984, [44, 1, 44]),
 (129081, [6718, 10691, 21365, 28540, 21369, 15368, 23462]),
 (474208, [2711, 1152, 1152, 1152, 4432, 1694]),
 (186623, [0, 14, 1]),
 (167777, [113, 113, 113, 113, 113, 113]),
 (156567, [542, 704, 915, 21468, 29785, 18479]),
 (301401, [3577, 943]),
 (483501, [9734, 9734]),
 (357004, [70, 10392]),
 (425968,
  [2195,
   3298,
   2193,
   2193,
   2194,
   2194,
   2194,
   3287,
   1079,
   1079,
   10025,
   10025,
   3628,
   3628,
   3178,
   3178,
   3180,
   3180]),
 (198374, [644, 2407, 4527, 4527, 4527, 708, 708, 10146, 10146, 21659]),
 (277044, [169, 169, 169]),
 (311987, [136, 153, 153, 153, 153, 136, 136, 153, 136, 136, 136, 136]),
 (2316, [903, 903, 903, 150, 150]),
 (327984,
  [346,
   346,
   269,
   269,
   5886,
   5239,
   11505,
   5239,
   5239,
   543,
   543,
   543,
   8232,
   2762,
   13126,
   17158]),
 (248566, [248, 248, 248]),
 (3599

In [29]:
def compute_score(train, target, itemId):
    candidate_sessions = []
    for s in range(len(train)):
        if itemId in train[s][1]:
            candidate_sessions.append(train[s][1])
    
    score = 0
    for n in range(len(candidate_sessions)):
        score += jaccard(candidate_sessions[n], target)
    
    return score

compute_score(train=train, target=target, itemId=1125)

0.0

In [30]:
categories = subset.loc[subset.sessionId==actual_session[0]]['category_code'].unique().tolist()
candidate_items = subset.loc[subset.category_code.isin(categories)]['itemId'].unique().tolist()
len(candidate_items)

1190

In [31]:
ranking = []
for i in range(len(candidate_items)):
    ranking.append((compute_score(train, target, candidate_items[i]), candidate_items[i]))

ranking.sort()
ranking.reverse()
print(ranking[0:10])

[(40.75125811798088, 1796), (5.7722157513303, 479), (4.972708706109319, 341), (4.702738420007047, 598), (3.734736884827183, 340), (3.350308433490894, 887), (3.3360429855499603, 274), (3.143141852996393, 248), (2.7870275657457544, 4338), (2.641333711214491, 3104)]


In [32]:
subset.loc[subset.itemId==21083]

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
262378,2019-10-28 11:56:34 UTC,view,21408343,electronics.clocks,romanson,02880533-d7ce-45bc-bc70-5d348203de11,21083,63162
329712,2019-10-11 15:52:48 UTC,view,21408343,electronics.clocks,romanson,032e7ca0-c78a-4167-b40b-e695e86a2f03,21083,79442
361345,2019-10-08 18:25:15 UTC,view,21408343,electronics.clocks,romanson,037aa64f-9798-4a10-b692-6d60d757ba6b,21083,87016
493880,2019-10-30 13:46:25 UTC,view,21408343,electronics.clocks,romanson,04c22360-b6c7-47bb-af50-501b85ce3399,21083,119192
657797,2019-10-08 17:07:25 UTC,view,21408343,electronics.clocks,romanson,0658adfb-8f43-4869-8f68-f11e16a40e45,21083,158819
667941,2019-10-09 07:40:36 UTC,view,21408343,electronics.clocks,romanson,06725817-89ff-4b8f-8c1a-adda02bd13a8,21083,161355
714952,2019-10-15 16:32:32 UTC,view,21408343,electronics.clocks,romanson,06e60861-aecc-4544-866b-a01499926aae,21083,172626
714953,2019-10-15 16:33:16 UTC,view,21408343,electronics.clocks,romanson,06e60861-aecc-4544-866b-a01499926aae,21083,172626
909150,2019-10-19 01:05:03 UTC,view,21408343,electronics.clocks,romanson,08c3c7ba-3874-45a7-a836-5952a02ed413,21083,219468
1055780,2019-10-07 10:35:35 UTC,view,21408343,electronics.clocks,romanson,0a3039ef-a7d3-4101-959b-30003cb161d4,21083,255199


***Exercício 01:*** A função compute_score() definida acima e explicada na aula, é a implementação do algoritmo Session-based KNN (S-KNN). Implemente uma variação da função que represente o algoritmo Sequential Session-based KNN (S-SKNN). Compare o desempenho de ambas as funções na recomendação do último item de uma sessão qualquer do conjunto de teste. Para fazer essa comparação, utilize a métrica Average Precision. 

In [119]:
def compute_score_ssknn(train, target, itemId):
    candidate_sessions = []
    for s in range(len(train)):
        if itemId in train[s][1]:
            candidate_sessions.append(train[s][1])
    
    score = 0
    w_n = 1
    for n in range(len(candidate_sessions)):
        candidate_n = candidate_sessions[n]
        candidate_itens_in_current_session = [item for item in candidate_n if item in target]
        if len(candidate_itens_in_current_session) > 0:
            most_recent = candidate_itens_in_current_session[-1]
            position_in_target = target.index(most_recent) + 1
            w_n = position_in_target / len(target)
        else:
            w_n = 0
        score += w_n * jaccard(candidate_sessions[n], target)
    
    return score
compute_score_ssknn(train=train, target=target, itemId=1125)

0.7263378692955686

In [57]:
categories = subset.loc[subset.sessionId==actual_session[0]]['category_code'].unique().tolist()
candidate_items = subset.loc[subset.category_code.isin(categories)]['itemId'].unique().tolist()
candidate_items

[176,
 192,
 193,
 194,
 195,
 205,
 249,
 300,
 301,
 302,
 303,
 304,
 349,
 353,
 396,
 453,
 454,
 561,
 750,
 751,
 752,
 753,
 791,
 792,
 793,
 794,
 795,
 796,
 797,
 798,
 799,
 800,
 801,
 802,
 998,
 999,
 1000,
 1001,
 1002,
 1082,
 1092,
 1123,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1149,
 1152,
 1320,
 1321,
 1322,
 1323,
 1400,
 1419,
 1420,
 1463,
 1464,
 1645,
 1646,
 1818,
 1843,
 1844,
 1906,
 1996,
 2074,
 2075,
 2099,
 2122,
 2123,
 2124,
 2125,
 2126,
 2127,
 2147,
 2148,
 2149,
 2255,
 2256,
 2257,
 2260,
 2262,
 2349,
 2503,
 2504,
 2505,
 2506,
 2507,
 2508,
 2509,
 2510,
 2511,
 2512,
 2513,
 2514,
 2576,
 2688,
 2708,
 2709,
 2710,
 2711,
 2806,
 2822,
 2829,
 2830,
 2846,
 2847,
 2879,
 2897,
 2920,
 2952,
 2960,
 2963,
 2964,
 2965,
 2966,
 2967,
 3026,
 3027,
 3066,
 3067,
 3068,
 3091,
 3093,
 3192,
 3220,
 3406,
 3579,
 3612,
 3613,
 3614,
 3615,
 3616,
 3617,
 3618,
 3619,
 3620,
 3621,
 3622,
 3623,
 3624,
 3842,
 3914,
 3915,


In [63]:
def precision_at_k(rec_list, ground_truth, k):
    focus_list = rec_list[:k]
    relevant_list = [1 if item in ground_truth else 0 for item in focus_list]
    n_relevants = sum(relevant_list)
    return n_relevants / k

In [65]:
def average_precision(recs_user, ground_truth):
    precisions = 0
    relevant_list = [1 if item in ground_truth else 0 for item in recs_user]
    n_relevants = sum(relevant_list)
    
    if (n_relevants == 0):
        return 0

    num_recs = len(recs_user)
    for i in range(num_recs):
        if (relevant_list[i] == 1):
            # only consider relevant documents
            k=i+1
            precision = precision_at_k(recs_user, ground_truth,k=k)
            precisions += precision

    return precisions/(n_relevants)


In [ ]:
actual_session = test[3]
target = actual_session[1][0:-1]
ground_truth = [ actual_session[1][-1] ]

In [ ]:
actual_session, target, ground_truth

In [74]:
def generate_top_k_rec(score_func, train, target, k):
    ranking = []
    for i in range(len(candidate_items)):
        ranking.append((score_func(train, target, candidate_items[i]), candidate_items[i]))
    ranking.sort()
    ranking.reverse()

    return ranking[0:k]

In [93]:
generate_top_k_rec(compute_score, train, target, 15)

[(0.09090909090909091, 193),
 (0.09090909090909091, 192),
 (0.08333333333333333, 14061),
 (0.08333333333333333, 2262),
 (0.06666666666666667, 2509),
 (0.045454545454545456, 998),
 (0, 42387),
 (0.0, 42252),
 (0.0, 42162),
 (0.0, 42161),
 (0.0, 41952),
 (0.0, 41949),
 (0.0, 41927),
 (0, 41926),
 (0, 41833)]

In [75]:
generate_top_k_rec(compute_score, train, target, 10)

[(15.112248258720983, 1400),
 (11.06543481381806, 1133),
 (3.1978208873091116, 6390),
 (3.108709304033853, 1149),
 (2.5519064685931023, 2846),
 (2.2038868140803123, 1152),
 (2.1655150236402756, 6212),
 (2.090660798833837, 2711),
 (2.0603824471301446, 9350),
 (1.898472506632925, 794)]

In [76]:
sknn_rec = generate_top_k_rec(compute_score, train, target, 10)
ssknn_rec = generate_top_k_rec(compute_score_ssknn, train, target, 10)

ap_sknn = average_precision(sknn_rec, ground_truth)
ap_ssknn = average_precision(ssknn_rec, ground_truth)

In [121]:
ap_sknn

0

In [122]:
ap_ssknn

0

Ambos algoritmos não foram capazes de listar o ground_truth nas top10 predições de maior valor. O que teve de diferença foi o ssknn possuir uma propensidade maior a recomendar itens que já estão presentes na sessão do usuário.


***Exercício 02:*** Implemente outra variação da função compute_score() que represente o algoritmo Sequential Filter Session-based KNN (SF-SKNN). Compare o desempenho desse algoritmo com as demais abordagens para uma sessão qualquer do conjunto de teste.

In [118]:
def compute_score_sfsknn(train, target, itemId, actual_session):
    last_item = actual_session[1][-1]
    candidate_sessions = []
    for s in range(len(train)):
        itens_in_session = train[s][1]
        if itemId in itens_in_session and last_item in itens_in_session:
            # Checa se os dois itens estão na sessão
            idx = itens_in_session.index(last_item)
            if idx+1 < len(items_in_session) and items_in_session[idx+1] == itemId:
                # Checa se o item posterior a last_item é o itemId alvo
                candidate_sessions.append(train[s][1])
            else:
                pass
    
    score = 0
    for n in range(len(candidate_sessions)):
        score += jaccard(candidate_sessions[n], target)
    
    return score

compute_score_sfsknn(train=train, target=target, itemId=1125, actual_session=actual_session)

0

***Exercício 03:*** Na aula utilizamos uma estratégia trivial para selecionar itens candidatos para poder calcular seu escore: selecionamos apenas itens da mesma categoria que os itens que estão na sessão atual. Isso pode ser um problema, pois numa sessão, um usuário pode estar visualizando um produto e o sistema poderia recomendar um produto de outra categoria (exemplo: usuário visualiza/compra um smartphone, e o sistema recomenda uma capa protetora). Pense e implemente uma estratégia para selecionar os itens candidatos para os quais será calculado o escore via função compute_score(). Lembre-se de que quanto menor a quantidade de itens candidatos, mais rápido o sistema irá gerar a recomendação top N. Explique sua estratégia.

R.: A estratégia abordada em sala é promissora, mas, como apontado, limita navegações entre categorias. Algo que podemos fazer é selecionar sessões de usuários similares obtidos por meio de um UserKNN. Isso garante candidatos similares ao usuário sem restringir navegação em uma única categoria. Porém, não possuimos informação de userId na base, ou em sistemas baseados em sessão no geral. O que podemos fazer, então, é uma abordagem baseada parcialmente em contexto, selecionando sessões realizadas na mesma semana.


Vamos primeiro entender qual a duração média de uma sessão

In [34]:
subset

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,0,0
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c,2,0
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,3,0
...,...,...,...,...,...,...,...,...
2000001,2019-10-29 15:36:18 UTC,view,1004659,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,210,483507
2000002,2019-10-29 15:36:22 UTC,view,1004659,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,210,483507
2000003,2019-10-29 15:36:42 UTC,view,1004659,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,210,483507
2000004,2019-10-29 15:37:44 UTC,view,1005014,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,211,483507


In [46]:
subset["event_timestamp"] = pd.to_datetime(subset["event_time"], format='%Y-%m-%d %H:%M:%S %Z')

In [47]:
def session_size(session_series):
    max_time = session_series.max()
    min_time = session_series.min()
    return max_time - min_time



In [49]:
grouped_sessions = subset.loc[:, ["event_timestamp", "user_session"]].groupby("user_session").agg({"event_timestamp": session_size})

In [52]:
def convert_to_minutes(timedelta):
    total_minutes = timedelta.components.days * 24 * 60 + timedelta.components.hours * 60 + timedelta.components.minutes
    return total_minutes

In [55]:
grouped_sessions["minutes"] = grouped_sessions["event_timestamp"].apply(convert_to_minutes)

In [56]:
grouped_sessions

,event_timestamp,minutes
user_session,,
00000056-a206-40dd-b174-a072550fa38c,0 days 00:08:13,8
00000083-8816-4d58-a9b8-f52f54186edc,0 days 00:09:45,9
000001fd-1f89-45e8-a3ce-fe3218cabfad,0 days 00:08:29,8
000003eb-b63e-45d9-9f26-f229057c654a,0 days 00:00:00,0
0000047e-bdcc-4854-9e8d-9da7f84010ae,0 days 00:01:04,1
...,...,...
134b577a-63ba-4d2f-a776-827be94c1bd2,0 days 00:00:00,0
134b57ed-e03e-4ff1-aaec-0e5b40dfb04b,0 days 00:05:04,5
134b5c96-4c37-4296-b893-12948f1dd667,0 days 00:00:00,0


In [59]:

grouped_sessions["minutes"].describe()

count    483508.000000
mean         15.988255
std         357.570629
min           0.000000
25%           0.000000
50%           0.000000
75%           4.000000
max       43290.000000
Name: minutes, dtype: float64

In [61]:
grouped_sessions[grouped_sessions["minutes"] > 0]["minutes"].describe()

count    236529.000000
mean         32.682880
std         510.702687
min           1.000000
25%           2.000000
50%           4.000000
75%           9.000000
max       43290.000000
Name: minutes, dtype: float64

Note que a maioria das sessões dura menos de dez minutos. Essa informação nos permite definir a granularidade de qual período de tempo vamos selecionar para atribuir sessões candidatas. Vamos, então, selecionar sessões que aconteceram no mesmo horário, porém, na semana anterior. Fazemos isso para tentar capturar tendências de compra similar em outro intervalo de tempo comparável.

Para acelerar o calculo, podemos descartar sessões de menos de 1 minuto

In [62]:
subset

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId,event_timestamp
0,2019-10-31 06:23:12 UTC,view,1005115,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,0,0,2019-10-31 06:23:12+00:00
1,2019-10-31 06:23:52 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0,2019-10-31 06:23:52+00:00
2,2019-10-31 06:25:30 UTC,view,1005105,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,1,0,2019-10-31 06:25:30+00:00
3,2019-10-31 06:26:58 UTC,view,1004858,electronics.smartphone,samsung,00000056-a206-40dd-b174-a072550fa38c,2,0,2019-10-31 06:26:58+00:00
4,2019-10-31 06:28:21 UTC,view,1005104,electronics.smartphone,apple,00000056-a206-40dd-b174-a072550fa38c,3,0,2019-10-31 06:28:21+00:00
...,...,...,...,...,...,...,...,...,...
2000001,2019-10-29 15:36:18 UTC,view,1004659,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,210,483507,2019-10-29 15:36:18+00:00
2000002,2019-10-29 15:36:22 UTC,view,1004659,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,210,483507,2019-10-29 15:36:22+00:00
2000003,2019-10-29 15:36:42 UTC,view,1004659,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,210,483507,2019-10-29 15:36:42+00:00
2000004,2019-10-29 15:37:44 UTC,view,1005014,electronics.smartphone,samsung,134b6774-58d5-461c-bcf1-a9005de7b819,211,483507,2019-10-29 15:37:44+00:00


In [66]:
def filter_timestamps(df, reference_timestamp):
    reference_time = pd.to_datetime(reference_timestamp)
    one_week_prior = reference_time - pd.Timedelta(weeks=1)
    start_time = one_week_prior - pd.Timedelta(minutes=10)
    end_time = one_week_prior + pd.Timedelta(minutes=10)
    
    return df[(df['event_timestamp'] >= start_time) & (df['event_timestamp'] <= end_time)]



In [68]:
candidate_for_session_0 = filter_timestamps(subset, subset.iloc[0]["event_time"])

In [69]:
candidate_for_session_0

,event_time,event_type,product_id,category_code,brand,user_session,itemId,sessionId,event_timestamp
9697,2019-10-24 06:19:11 UTC,view,1004256,electronics.smartphone,apple,001a0171-ee55-4844-927e-ff2204e44020,2409,2465,2019-10-24 06:19:11+00:00
16920,2019-10-24 06:29:08 UTC,view,1005118,electronics.smartphone,apple,002ce109-ee14-49fa-816e-edc1c672df64,159,4305,2019-10-24 06:29:08+00:00
18347,2019-10-24 06:19:14 UTC,view,1004669,electronics.smartphone,xiaomi,00303f28-8090-4ac9-beff-5c6411ce24bc,1796,4635,2019-10-24 06:19:14+00:00
18348,2019-10-24 06:21:40 UTC,view,1004669,electronics.smartphone,xiaomi,00303f28-8090-4ac9-beff-5c6411ce24bc,1796,4635,2019-10-24 06:21:40+00:00
18349,2019-10-24 06:22:20 UTC,view,1004892,electronics.smartphone,zte,00303f28-8090-4ac9-beff-5c6411ce24bc,1314,4635,2019-10-24 06:22:20+00:00
...,...,...,...,...,...,...,...,...,...
1999314,2019-10-24 06:26:36 UTC,view,1004856,electronics.smartphone,samsung,1349d041-7dc7-4cba-82fb-0c0ac9b0f76d,18,483341,2019-10-24 06:26:36+00:00
1999315,2019-10-24 06:27:37 UTC,cart,1004856,electronics.smartphone,samsung,1349d041-7dc7-4cba-82fb-0c0ac9b0f76d,18,483341,2019-10-24 06:27:37+00:00
1999316,2019-10-24 06:28:13 UTC,view,1004856,electronics.smartphone,samsung,1349d041-7dc7-4cba-82fb-0c0ac9b0f76d,18,483341,2019-10-24 06:28:13+00:00
1999317,2019-10-24 06:28:22 UTC,cart,1004856,electronics.smartphone,samsung,1349d041-7dc7-4cba-82fb-0c0ac9b0f76d,18,483341,2019-10-24 06:28:22+00:00


Temos aqui 1189 sessões que ocorreram em um horário similar, mas na semana anterior. Vamos comparar com a quantidade de itens comparados utilizando a metodologia utilizada em aula

In [70]:
categories = subset.loc[subset.sessionId==actual_session[0]]['category_code'].unique().tolist()
candidate_items = subset.loc[subset.category_code.isin(categories)]['itemId'].unique().tolist()
len(candidate_items)

1190

Coincidentemente, o número de sessões é basicamente igual. No entanto, aqui temos uma diversidade maior de categorias.